<a href="https://colab.research.google.com/github/schauppi/Udemy_TensorFlow_Developer/blob/main/Abschnitt_8_NLP_fundamentals_in_TensorFlow/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (text or speech sequences).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## check for GPU

In [104]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-512cf063-0d99-6e34-fcc5-9177b964bd20)


## Get helper functions

In [105]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-08-13 09:01:51--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-13 09:01:52 (60.6 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [106]:
# Import series of helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we´re going to be using is Kaggle´s introduction to BLP dataset (text samples of Tweets labelled as disaster or nor disaster)

In [107]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2021-08-13 09:01:52--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-08-13 09:01:52 (103 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



## Visualizing a text dataset 

To visualize our text samples, we first have to read them in, one way to do so would be to use Python.

Another way to do this is to use pandas...

In [108]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [109]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [110]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [111]:
#What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [112]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [113]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [114]:
# Let´s visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
RT @tonyhsieh: 'The person who dances with you in the rain will most likely walk with you in the storm.' -Anonymous

---

Target: 0 (not real disaster)
Text:
Beyonce Is my pick for http://t.co/nnMQlz91o9 Fan Army #Beyhive http://t.co/o91f3cYy0R 77

---

Target: 0 (not real disaster)
Text:
Ted Cruz fires back at Jeb &amp; Bush: ÛÏWe lose because of Republicans like Jeb &amp; Mitt.Û [Video] -  http://t.co/BFTHaHLCr0

---

Target: 1 (real disaster)
Text:
An IS group suicide bomber detonated an explosives-packed vest in a mosque inside a Saudi special forces headquarters killing 15 people.

---

Target: 0 (not real disaster)
Text:
Top link: Reddit's new content policy goes into effect many horrible subreddits banned or quarantined http://t.co/zCp5cszSLl

---



### Split data into training and validation sets

In [115]:
from sklearn.model_selection import train_test_split

#Use train_test_split to split train data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [116]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [117]:
#Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Convert text into numbers

When dealing with a text problem, one of the first things you´ll have to do before you can build a model is to convert your text to numbers.

There are a few ways you can to this, namely:
* Tokenization - direct mapping of token (a token could be a word or character to number)
* Embedding - create a matrix of a feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

## Text vectorization (tokenization)

In [118]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, #how man word in the vocabulary (automatically adds <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, #create groups of n-words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, #how long the sequences should be
                                    pad_to_max_tokens=True)

In [119]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [120]:
len(train_sentences[0].split())

7

In [121]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [122]:
#Setup vectorizeation variables
max_vocab_length = 10000 #max number of words to have in our vocabulary
max_length = 15 #max length our sequences will be (e.g. how many words from a tweet will our model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [123]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [124]:
#Create a sample sentence and tokenize it
sample_sentence = "There´s a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [125]:
# choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it!       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 546,   46, 1029,  151,  138,    5,  125,   54,   15,   53, 1636,
          15,   28, 2405,   34]])>

In [126]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] #get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(len(words_in_vocab)) 
print(top_5_words)
print(bottom_5_words)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we´re going to use TensorFlow´s embedding layer.

The parameters we care most about for our embedding layer:
* input_dim = the size of the vocabulary
* output_dim = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [127]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

In [128]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence} \
      \n\nEmbedded version")
#Embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 I always tell my mom to bring me food or I will hold her cat hostage??       

Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03761773, -0.0112257 ,  0.01476211, ..., -0.03036818,
          0.04098736,  0.03273804],
        [-0.02420437,  0.04045621, -0.01493071, ..., -0.02203947,
          0.03672654, -0.03010169],
        [-0.01374195,  0.03537961,  0.04202158, ...,  0.03439074,
          0.02029295, -0.01105816],
        ...,
        [-0.02919639,  0.00490249, -0.03459734, ..., -0.0243655 ,
          0.02161661, -0.01974881],
        [-0.00856047,  0.02351776, -0.02034231, ...,  0.0078366 ,
         -0.023106  ,  0.01324241],
        [ 0.0218753 , -0.01850209,  0.00225643, ..., -0.02669281,
          0.01481762, -0.01648034]]], dtype=float32)>

In [129]:
# Check out a single token´s embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-3.7617732e-02, -1.1225700e-02,  1.4762115e-02,  5.3932294e-03,
         3.7346710e-02,  7.0132315e-05,  3.8266990e-02, -4.7331896e-02,
        -1.1163391e-02,  3.5320017e-02, -3.1987384e-02,  1.2967970e-02,
         4.6969976e-02,  2.2026148e-02,  2.3454536e-02,  1.3531137e-02,
        -3.3764161e-02,  9.9448077e-03, -4.8340667e-02, -2.2862589e-02,
        -2.6580466e-02,  2.2990596e-02,  3.0739341e-02,  2.9189233e-02,
         2.4188582e-02, -4.1666280e-02,  3.6154632e-02, -2.4772063e-03,
         2.3927581e-02,  9.3980879e-04, -1.2301318e-03, -9.2371553e-04,
         8.0453642e-03, -2.2423780e-02, -4.4842400e-02,  3.0274902e-02,
        -3.5245251e-02,  2.7101945e-02, -3.8912356e-02,  3.5576154e-02,
        -1.9722974e-02,  1.0623656e-02, -2.1076132e-02, -1.8671285e-02,
         1.7127339e-02,  4.5612719e-02, -4.3930557e-02, -2.0642638e-02,
        -2.2073878e-02, -2.7460564e-02, -8.2296133e-03,  1.9185830e-02,
         5.8315

## Modelling a text dataset (running a series of experiments)

Now we´ve got a way to turn our text sequences into numbers, it´s time to start building a series of modelling experiments.

We´ll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM
* Model 3: GRU
* Model 4: Bidirectional LSTM
* Model 5: 1D Convolution NN
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data

How are we goint to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit the model
* Evaluate the model

### Model 0: Getting a baseline

In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokenization and modellin pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #cobvert words to numbers
                    ("clf", MultinomialNB()) #model the text
])

#Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [131]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [132]:
#Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

In [133]:
#Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 scroe of a binary classification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, racall and f1-scroe using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}

  return model_results

In [134]:
#Get baseline results
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [135]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [136]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) #inputs are 1-dimensional strings
x = text_vectorizer(inputs) #turn the input text into numbers
x = embedding(x) #create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) #create the feature vector for each token
outputs = layers.Dense(1, activation="sigmoid")(x) #Create the binary output
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [137]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [138]:
# Compile model
model_1.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [139]:
#Fit the model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210813-090156
Epoch 1/5
215/215 [==============================] - 5s 20ms/step - loss: 0.6124 - accuracy: 0.6987 - val_loss: 0.5372 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4409 - accuracy: 0.8200 - val_loss: 0.4690 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.3459 - accuracy: 0.8597 - val_loss: 0.4614 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2835 - accuracy: 0.8902 - val_loss: 0.4647 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2368 - accuracy: 0.9132 - val_loss: 0.4771 - val_accuracy: 0.7808


In [140]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4771 - accuracy: 0.7808


[0.47708848118782043, 0.7808399200439453]

In [141]:
#Make some predictions and evaluate those
model_1_preds_probs = model_1.predict(val_sentences)
model_1_preds_probs.shape

(762, 1)

In [142]:
#look at a single prediction
model_1_preds_probs[0]

array([0.4013793], dtype=float32)

In [143]:
#look at the first 10 predictions
model_1_preds_probs[:10]

array([[0.4013793 ],
       [0.74617606],
       [0.9976446 ],
       [0.1371503 ],
       [0.10724574],
       [0.93752396],
       [0.91679657],
       [0.99386567],
       [0.9741527 ],
       [0.2912919 ]], dtype=float32)

In [144]:
#Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_preds_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [145]:
# Calculate our model_1 results
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results

{'accuracy': 78.08398950131233,
 'f1': 0.7783998521836788,
 'precision': 0.783783808499639,
 'recall': 0.7808398950131233}

In [146]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [147]:
# Get the vocabulary from the text vectorization
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [148]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [150]:
#Get weight matrix of embedding layer
#these are the numerical representations of each token in our training data
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape) #same size as vocav size and embedding dim

(10000, 128)


To visualize, TensorFlow has a handy tool calles projector : https://projector.tensorflow.org/

In [151]:
#Create embedding files

import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [152]:
#Download files from colab to upload to projector
"""
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
"""

"\ntry:\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception:\n  pass\n"

## Recurrent Neural Networks (RNN´s)

The premise of a RNN is to use the representation of a previous input to aid the representation of the later input.

### Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typicall looks like this:

Input (text) - Tokenize - Embedding - Layers (RNN/dense) - Output (label probability)

In [153]:
#Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.LSTM(64, return_sequences=True)(x) #when you´re stacking RNN cells together, you need to set return_sequences=True
x = layers.LSTM(64, return_sequences=False)(x)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [154]:
#Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [155]:
#Compile the model
model_2.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [156]:
#Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210813-090759
Epoch 1/5
215/215 [==============================] - 8s 24ms/step - loss: 0.2231 - accuracy: 0.9196 - val_loss: 0.5670 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.1550 - accuracy: 0.9426 - val_loss: 0.7146 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.1284 - accuracy: 0.9524 - val_loss: 0.7506 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.1026 - accuracy: 0.9594 - val_loss: 0.7594 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0851 - accuracy: 0.9663 - val_loss: 0.9132 - val_accuracy: 0.7756


In [157]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[1.8792320e-03],
       [8.2653737e-01],
       [9.9949861e-01],
       [3.1141406e-02],
       [3.3108320e-04],
       [9.6556139e-01],
       [5.3630626e-01],
       [9.9960858e-01],
       [9.9934000e-01],
       [2.1382971e-01]], dtype=float32)

In [158]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [159]:
#Calculate model 2 results
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'f1': 0.770997050538307,
 'precision': 0.7841484675632434,
 'recall': 0.7755905511811023}

In [160]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [161]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=False)(x)
#x = layers.LSTM(64)(x)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [162]:
#get model summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                37248     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [163]:
#Compile the model
model_3.compile(loss="binary_crossentropy", optimizer="Adam", metrics="accuracy")

In [164]:
#Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20210813-090844
Epoch 1/5
215/215 [==============================] - 7s 24ms/step - loss: 0.1598 - accuracy: 0.9385 - val_loss: 0.8298 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0833 - accuracy: 0.9693 - val_loss: 0.8006 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0693 - accuracy: 0.9734 - val_loss: 0.9033 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0610 - accuracy: 0.9740 - val_loss: 1.0746 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0514 - accuracy: 0.9764 - val_loss: 1.1617 - val_accuracy: 0.7743


In [165]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.1937498e-02],
       [7.5406992e-01],
       [9.9963033e-01],
       [1.7922531e-01],
       [1.4687696e-04],
       [9.9965656e-01],
       [7.8835469e-01],
       [9.9987900e-01],
       [9.9980968e-01],
       [9.5342684e-01]], dtype=float32)

In [166]:
#Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [167]:
#Calculate model 3 results
model_3_results = calculate_results(val_labels, model_3_preds)
model_3_results

{'accuracy': 77.42782152230971,
 'f1': 0.773426649133647,
 'precision': 0.7741317342942483,
 'recall': 0.7742782152230971}

### Model 4: Bidirectional RNN

Normal RNN´s go from left to right (just like you´d read an english sentence) however, a bidirectional RNN goes from right to left as well as from right to left.

In [168]:
# Build a bidirectional RNN in tensorflow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [169]:
#Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [170]:
#Compile the model
model_4.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [171]:
#Fit the model
model_4_history = model_4.fit(train_sentences, 
                              train_labels, 
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210813-090928
Epoch 1/5
215/215 [==============================] - 11s 34ms/step - loss: 0.1036 - accuracy: 0.9704 - val_loss: 1.0226 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0513 - accuracy: 0.9771 - val_loss: 1.3406 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0445 - accuracy: 0.9794 - val_loss: 1.2593 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0440 - accuracy: 0.9804 - val_loss: 1.5102 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0373 - accuracy: 0.9818 - val_loss: 1.4781 - val_accuracy: 0.7690


In [172]:
# Make predictions
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[7.1779272e-04],
       [7.5630802e-01],
       [9.9995089e-01],
       [2.6012307e-01],
       [5.4305665e-06],
       [9.9946493e-01],
       [2.8271589e-01],
       [9.9998581e-01],
       [9.9997747e-01],
       [9.9805272e-01]], dtype=float32)

In [173]:
#Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [174]:
#calculate the result of our bidirectional model
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 76.9028871391076,
 'f1': 0.7669342344352704,
 'precision': 0.7706028054440214,
 'recall': 0.7690288713910761}

## Convolution Neural Networks for Text (and other types of sequences)

We´ve used CNNs for images but images are typically 2D (height x width)... however, out text data is 1D.

Previously we´ve used Conv2D for our image data but now we´re going to use Conv1D.

The typical structure of a Conv1D model for sequences (in out case, text):

Inputs (text) -> Tokenization -> Embedding -> Layers (Conv1D + Pooling) -> Outputs (class probabilities)

### Model 5: Conv1D

For different explanations of parameter see:
* CNN explainer (this is for 2D but can relate to 1D)
* Difference between same and valid - Google

In [175]:
# Test out out embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) #turn target into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, #looks at 5 words at a time
                        activation="relu",
                        padding="valid") # deafult = "valid", the output is smaller than the input shape

conv_1d_output = conv_1d(embedding_test) #pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output) #get the most important features - features with highest values

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [176]:
#embedding_test

In [177]:
#conv_1d_output

In [178]:
#max_pool_output

In [179]:
# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  activation="relu",
                  padding="valid",
                  strides=1)(x)
x = layers.GlobalMaxPooling1D()(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [180]:
#Compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer="Adam", 
                metrics=["accuracy"])

In [181]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [182]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20210813-091001
Epoch 1/5
215/215 [==============================] - 5s 20ms/step - loss: 0.1284 - accuracy: 0.9565 - val_loss: 0.8619 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0744 - accuracy: 0.9708 - val_loss: 1.0324 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0627 - accuracy: 0.9742 - val_loss: 1.1407 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0569 - accuracy: 0.9768 - val_loss: 1.1460 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0514 - accuracy: 0.9778 - val_loss: 1.2308 - val_accuracy: 0.7507


In [183]:
# Make some predictions
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[1.20458744e-01],
       [7.09752560e-01],
       [9.99951959e-01],
       [6.37307540e-02],
       [4.61782612e-08],
       [9.95769620e-01],
       [9.47468996e-01],
       [9.99904633e-01],
       [9.99999642e-01],
       [8.99997950e-01]], dtype=float32)

In [184]:
#Convert pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [185]:
#Evaluate predictions
model_5_results = calculate_results(val_labels,
                                    model_5_preds)
model_5_results

{'accuracy': 75.06561679790026,
 'f1': 0.7491705324008289,
 'precision': 0.7507272180149456,
 'recall': 0.7506561679790026}

## Model 6: TensorFlow Hub Pretrained Sentence Encoder

Now we´ve build a few of our own models, let´s try and use transfer learning for NLP, specifically using TensorFlow Hub´s pretrained Encoder: https://tfhub.dev/google/universal-sentence-encoder/4

In [186]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed(["sample sentence", "When you call the universal sentence encoder, it turns it into number"])
embed_samples[0][:50]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([-0.03938255,  0.00283756,  0.02243922,  0.0245115 , -0.04985989,
        0.05275185,  0.01906133,  0.03593649, -0.04484411,  0.01787181,
       -0.03389932, -0.00872529, -0.02777038,  0.09363514, -0.02399772,
       -0.08983202, -0.03871281,  0.00450141, -0.02363318, -0.03881208,
       -0.01647638,  0.04881011, -0.00920218,  0.05666861, -0.06821936,
        0.06926924, -0.01130831, -0.0776007 ,  0.00864499, -0.02895822,
        0.00346943,  0.01833375, -0.06229274,  0.02653736, -0.09129825,
        0.02586216, -0.00045973,  0.03273476, -0.06651258,  0.03915777,
       -0.03467299,  0.0575624 , -0.00661325, -0.01383738, -0.04131497,
       -0.02762501,  0.04249629, -0.00791721,  0.04456056, -0.01393928],
      dtype=float32)>

In [187]:
embed_samples[0].shape

TensorShape([512])

In [188]:
# Create a Keras Layers using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], #because input can be of variable length
                                        dtype=tf.string, trainable=False, name="USE")

In [198]:
# Build the model

model_6 = tf.keras.Sequential([
                               sentence_encoder_layer,
                               layers.Dense(64, activation="relu"),
                               layers.Dense(1, activation="sigmoid")
                               
], name="model_6_USE")

In [199]:
#Compile the model
model_6.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [200]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_12 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [201]:
# Fit the model
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20210813-093414
Epoch 1/5
215/215 [==============================] - 6s 24ms/step - loss: 0.5060 - accuracy: 0.7854 - val_loss: 0.4496 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.4154 - accuracy: 0.8139 - val_loss: 0.4410 - val_accuracy: 0.8071
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4009 - accuracy: 0.8206 - val_loss: 0.4366 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3932 - accuracy: 0.8247 - val_loss: 0.4365 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3848 - accuracy: 0.8311 - val_loss: 0.4248 - val_accuracy: 0.8189


In [202]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.17159581],
       [0.82918555],
       [0.99143267],
       [0.21850964],
       [0.74741983],
       [0.7336558 ],
       [0.9840274 ],
       [0.98336697],
       [0.9511763 ],
       [0.10333898]], dtype=float32)

In [203]:
#Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [204]:
#Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 81.88976377952756,
 'f1': 0.817984880977007,
 'precision': 0.8196605460013572,
 'recall': 0.8188976377952756}

In [205]:
len(train_sentences)

6851

## Model 7: TF Hub Pretrained USE but with 10% of training data

Transfer learning helps when you don´t have a large dataset.
To see how our model performs on a smaller dataset, let´s replicate model_6 except we´ll train it on 10% of the data.

In [209]:
# Create subsets of 10% of the training data
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()

In [210]:
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [211]:
#Check the number of targets in our subset of data
train_10_percent["target"].value_counts()

0    413
1    348
Name: target, dtype: int64

In [214]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

To recreate a model same to a previous model you´ve created you can use the tf.keras.models.clone_model() method 

In [215]:
#Build the model - same as model_6

model_7 = tf.keras.models.clone_model(model_6)

In [216]:
# Compile the model
model_7.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [217]:
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_12 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [218]:
#Fit the model
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "tf_hub_sentence_encoder_10_percent")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent/20210813-095335
Epoch 1/5
24/24 [==============================] - 6s 135ms/step - loss: 0.6628 - accuracy: 0.7188 - val_loss: 0.6221 - val_accuracy: 0.7966
Epoch 2/5
24/24 [==============================] - 1s 45ms/step - loss: 0.5886 - accuracy: 0.7858 - val_loss: 0.5429 - val_accuracy: 0.8005
Epoch 3/5
24/24 [==============================] - 1s 43ms/step - loss: 0.5159 - accuracy: 0.7976 - val_loss: 0.4788 - val_accuracy: 0.8058
Epoch 4/5
24/24 [==============================] - 1s 30ms/step - loss: 0.4625 - accuracy: 0.8081 - val_loss: 0.4371 - val_accuracy: 0.8176
Epoch 5/5
24/24 [==============================] - 1s 30ms/step - loss: 0.4285 - accuracy: 0.8160 - val_loss: 0.4085 - val_accuracy: 0.8281


In [219]:
# Make predictions 
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.23691262],
       [0.8167402 ],
       [0.91525126],
       [0.33503708],
       [0.7688092 ],
       [0.8158242 ],
       [0.901649  ],
       [0.93976486],
       [0.8272687 ],
       [0.06308784]], dtype=float32)

In [221]:
#turn pred probs into labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [222]:
#Evaluate predictions
model_7_results = calculate_results(val_labels, model_7_preds)
model_7_results

{'accuracy': 82.80839895013123,
 'f1': 0.8272554155579622,
 'precision': 0.8288722528604506,
 'recall': 0.8280839895013123}